In [ ]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=0T2I8H7HV0bFjWr9FjsNQTSvn0dvBP&access_type=offline&code_challenge=7p8RbFQv6dnQqNkK6YLVHy0WeVsZheJT7sD7WoHksK8&code_challenge_method=S256


You are now logged in as [nbenyoussef11@gmail.com].
Your current project is [aihealthbot].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



In [5]:
%pip install -q google-cloud-storage unstructured langchain python-magic sqlalchemy langchain_google_cloud_sql_pg


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import storage
# Initialize a client
client = storage.Client()
# Access the bucket
bucket_name = 'aihealthbot-bucket'
bucket = client.get_bucket(bucket_name)
# List all files in the 'data' directory
blobs = bucket.list_blobs(prefix="data/")
print("Files in 'dauphine-bucket/data':")
for blob in blobs:
    print(blob.name)

Files in 'dauphine-bucket/data':
data/
data/medquad.csv


In [8]:
file_path = 'data/medquad.csv'
# Get the blob object
blob = bucket.get_blob(file_path)
if blob:
    print(f"Information for '{file_path}':")
    print(f"Size: {blob.size} bytes")
    print(f"Content Type: {blob.content_type}")
    formatted_updated_on = (blob.updated).strftime("%Y-%m-%d %H:%M:%S") 
    print(f"Updated On: {formatted_updated_on}")
    print(f"Blob name: {blob.name}")
else:
    print(f"File '{file_path}' not found in the bucket.")

Information for 'data/medquad.csv':
Size: 22835609 bytes
Content Type: text/csv
Updated On: 2025-02-11 19:16:27
Blob name: data/medquad.csv


In [3]:
from langchain_core.documents import Document
import os
from google.cloud.storage.bucket import Bucket
from langchain_unstructured import UnstructuredLoader
import pandas as pd

In [ ]:
DOWNLOADED_LOCAL_DIRECTORY = "./downloaded_files"
os.makedirs(DOWNLOADED_LOCAL_DIRECTORY, exist_ok=True)

# Télécharger le fichier
def download_file_from_bucket(bucket, file_path: str) -> str:
    blob = bucket.blob(file_path)
    local_file_name = os.path.basename(file_path)
    local_filepath = os.path.join(DOWNLOADED_LOCAL_DIRECTORY, local_file_name)
    blob.download_to_filename(local_filepath)
    print(f"Downloaded '{file_path}' to '{local_file_name}'")
    return local_filepath

In [8]:
file_path = "data/medquad.csv"
local_filepath = download_file_from_bucket(bucket, file_path)

Downloaded 'data/medquad.csv' to 'medquad.csv'


In [48]:
CSV_FILE_PATH = os.path.join(DOWNLOADED_LOCAL_DIRECTORY, "medquad.csv")
df = pd.read_csv(CSV_FILE_PATH)

In [49]:
print(f"Nombre de lignes dans medquad.csv : {len(df)}")

Nombre de lignes dans medquad.csv : 16412


In [50]:
df['answer'] = df['answer'].fillna("Pas de réponse disponible")
df['source'] = df['source'].fillna("Source inconnue")
df['focus_area'] = df['focus_area'].fillna("Non catégorisé")

assert not df.isnull().values.any(), "Il reste des NaN dans le DataFrame !"

In [ ]:
print(df.isnull().sum())

question      0
answer        0
source        0
focus_area    0
dtype: int64


In [55]:
# Convertir chaque ligne du CSV en `Document` LangChain
documents = [
    Document(
        page_content=row["question"],  
        metadata={
            "answer": row["answer"],
            "source": row["source"],
            "focus_area": row["focus_area"]
        }
    )
    for _, row in df.iterrows()
]
print(f"{len(documents)} documents convertis avec succès !")

16412 documents convertis avec succès !


In [52]:
# Afficher le premier document
print("\n🔍 Premier document :")
print(documents[0])  # Affiche tout l'objet Document

# Afficher uniquement son contenu et ses métadonnées
print("\n📜 Question :", documents[0].page_content)
print("💡 Réponse :", documents[0].metadata["answer"])
print("📌 Source :", documents[0].metadata["source"])
print("📂 Focus Area :", documents[0].metadata["focus_area"])


🔍 Premier document :
page_content='What is (are) Glaucoma ?' metadata={'answer': "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye an

------

In [13]:
%pip install --upgrade --quiet  langchain-google-cloud-sql-pg langchain-google-vertexai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [85]:
from dotenv import load_dotenv
load_dotenv()

True

In [86]:
import os
from config import PROJECT_ID, REGION, INSTANCE, DATABASE, DB_USER
DB_PASSWORD = os.environ["DB_PASSWORD"]

In [88]:
TABLE_NAME = "nb_table"

In [89]:
from langchain_google_cloud_sql_pg import PostgresEngine
engine = PostgresEngine.from_instance(
    project_id=PROJECT_ID,
    instance=INSTANCE,
    region=REGION,
    database=DATABASE,
    user=DB_USER,
    password=DB_PASSWORD,
)

In [90]:
from sqlalchemy.exc import ProgrammingError
try:
    await engine.ainit_vectorstore_table(
        table_name=TABLE_NAME,
        vector_size=768,
    )
except ProgrammingError:
    print("Table already created")

In [91]:
#2eme execution pour verifier la creation de table
from sqlalchemy.exc import ProgrammingError
try:
    await engine.ainit_vectorstore_table(
        table_name=TABLE_NAME,
        vector_size=768,
    )
except ProgrammingError:
    print("Table already created")

Table already created


In [92]:
from langchain_google_vertexai import VertexAIEmbeddings
embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest",
    project=PROJECT_ID
)
print("✅ Connexion à Vertex AI réussie !")

✅ Connexion à Vertex AI réussie !


In [ ]:
from langchain_google_cloud_sql_pg import PostgresVectorStore
vector_store = PostgresVectorStore.create_sync( 
    engine=engine,
    table_name=TABLE_NAME,
    embedding_service=embedding,
)
# Convertir les documents en embeddings et les stocker dans Cloud SQL
'''vector_store.add_documents(
    [
        Document(
            page_content=row["question"], 
            metadata={
                "answer": row["answer"],
                "source": row["source"],
                "focus_area": row["focus_area"]
            }
        )
        for _, row in df.iterrows()
    ]
)
print("Les embeddings des questions ont été générés et stockés dans la table !")'''

✅ Les embeddings des questions ont été générés et stockés dans la table !


In [98]:
query = "What is Alzheimer ? " 

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.5} 
)

In [100]:
docs = retriever.invoke(query)

In [104]:
# Afficher les résultats
if docs:
    print(f"\n {len(docs)} documents trouvés pour la requête : {query}")
    for i, doc in enumerate(docs):
        print("-" * 50)
        print(f" Résultat {i+1} ")
        print(" Question trouvée: ", doc.page_content)
        print(" Réponse:", doc.metadata["answer"])
        print(" Source:", doc.metadata["source"])
        print(" Focus Area:", doc.metadata["focus_area"])
else:
    print("Aucun document pertinent trouvé pour cette requête.")


 4 documents trouvés pour la requête : What is Alzheimer ? 
--------------------------------------------------
 Résultat 1 
 Question trouvée:  What is (are) Alzheimer disease ?
 Réponse: Alzheimer disease is a degenerative disease of the brain that causes dementia, which is a gradual loss of memory, judgment, and ability to function. This disorder usually appears in people older than age 65, but less common forms of the disease appear earlier in adulthood.  Memory loss is the most common sign of Alzheimer disease. Forgetfulness may be subtle at first, but the loss of memory worsens over time until it interferes with most aspects of daily living. Even in familiar settings, a person with Alzheimer disease may get lost or become confused. Routine tasks such as preparing meals, doing laundry, and performing other household chores can be challenging. Additionally, it may become difficult to recognize people and name objects. Affected people increasingly require help with dressing, eating,